In [1]:
import pandas as pd
import numpy as np
import json
import random
import math
from math import radians, cos, sin, asin, sqrt

In [ ]:
def compute_dis(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(float, [lon1, lat1, lon2, lat2])
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine公式
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    r = 6371 # 地球平均半径，单位为公里
    return c * r * 1000

def dis_from_center(oc, dc, cc):
    o_lng, o_lat = oc.split(',')
    d_lng, d_lat = dc.split(',')
    co_lng, co_lat, cd_lng, cd_lat = cc.split(',')
    return compute_dis(o_lng, o_lat, co_lng, co_lat) + compute_dis(d_lng, d_lat, cd_lng, cd_lat)
    
def dis_diff(g):
    result = []
    for i in g.index:
        oc, dc, cc = g.ix[i, ['oc', 'dc', 'cc']]
        result.append(dis_from_center(oc, dc, cc))
    
    return [np.std(result), np.mean(result), np.max(result)]

In [ ]:
top = pd.read_csv("top_20170519_2.txt", delimiter="~", names=["o", "d", "oc", "dc", "dis", "uv"])

In [ ]:
len(top)

In [ ]:
top.sort_values('uv', ascending=False)

In [ ]:
from scipy.cluster.vq import kmeans2
lines = []
for i in top.index:
    oc = top.ix[i]['oc']
    dc = top.ix[i]['dc']
    uv = top.ix[i]['uv']
    lines.append([float(c) for c in oc.split(',')] + [float(c) for c in dc.split(',')])
lines = np.array(lines)

## KMEANS

In [379]:
centers, labels = kmeans2(lines, 1000, iter=100, minit='points')

In [380]:
clustered = pd.concat((top, 
    pd.DataFrame(labels, columns=['cluster']), 
    pd.DataFrame(np.array([','.join([str(c) for c in list(centers[i])]) for i in labels]), 
                 columns=['cc'])), axis=1)

## DBSCAN

In [92]:
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets.samples_generator import make_blobs
from sklearn.preprocessing import StandardScaler

In [93]:
def dis_between_od(x, y):
    return compute_dis(x[0], x[1], y[0], y[1]) + compute_dis(x[2], x[3], y[2], y[3])

In [94]:
db = DBSCAN(eps=1000, min_samples=2, metric=dis_between_od).fit(lines)
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_

In [95]:
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_clusters_, np.sum(core_samples_mask), np.sum(labels==-1)

(123, 331, 839)

In [96]:
clustered = pd.concat((top, 
    pd.DataFrame(labels, columns=['cluster'])), axis=1)

In [97]:
def map_cluster(r):
    return r['seq'] + n_clusters_ if r['cluster'] == -1 else r['cluster']
clustered['seq'] = range(len(clustered))
clustered['cluster'] = clustered.apply(map_cluster, axis=1)
del clustered['seq']

## 根据聚类结果进行统计

In [98]:
label_count_dict = dict(pd.DataFrame(labels)[0].value_counts())

In [99]:
clustered['cluster_count'] = clustered['cluster'].map(label_count_dict)

In [100]:
def top_uv(df, n=1, column='uv'):
    return df.sort_index(by=column)[-n:]
cluster_top = clustered.groupby('cluster', group_keys=False).apply(top_uv)
cluster_top['cc'] = cluster_top.oc + ',' + cluster_top.dc
cluster_top = cluster_top[['cluster', 'o', 'd', 'cc']]

/root/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:2: FutureWarning: by argument to sort_index is deprecated, pls use .sort_values(by=...)
  from ipykernel import kernelapp as app


In [101]:
clustered = pd.merge(clustered, cluster_top, on = 'cluster')

In [102]:
cluster_stats = clustered.groupby('cluster', as_index=False)['uv'].agg(['mean', 'sum']).add_prefix('uv_')

## 统计聚类内的距离

In [103]:
cluster_dis = clustered.groupby('cluster', group_keys=False).apply(lambda g: dis_diff(g))

dis_stats = []
for c in cluster_dis.index:
    dis_stats.append(cluster_dis.ix[c] + [c])
cluster_dis_stats = pd.DataFrame(dis_stats, columns=["dis_std", "dis_mean", "dis_max", "cluster"])

## join得到最终结果

In [104]:
complete = pd.merge(clustered, cluster_stats, left_on = 'cluster', right_index=True)
complete = pd.merge(complete, cluster_dis_stats, on='cluster')
# complete = pd.merge(complete, cluster_top, left_on='cluster', right_on='cluster')

In [105]:
complete.to_csv('cluster.csv', sep="\t", index=False)
from IPython.display import FileLink
FileLink('cluster.csv')

/home/zhli7/bus/cluster.csv

## Top

In [106]:
result = []
offset = 0
top_count = 10
top_clusters = complete.sort_values('uv_sum', ascending=False).cluster.unique()[offset:offset+top_count]
color_map = dict(zip(top_clusters[:top_count], 
    ['rgba(' + str(int(random.random() * 255)) + ', ' + str(int(random.random() * 255)) + ', ' + str(int(random.random() * 255)) + ', 0.8)' for i in range(top_count)]))

for l in top_clusters:
#     count = np.sum(top.values[labels==l, 5])
#     center = centers[l]
    represent = complete[complete.cluster==l].sort_index(by='uv', ascending=False)[-1:]
    count = complete[complete.cluster==l].uv.sum()
    center = represent.oc.values[0].split(',') + represent.dc.values[0].split(',')
    if count > 0:
        result.append({"geometry":{"type":"LineString","coordinates":
                [list(center[0:2]), list(center[2:])]},"count":count,
                      "strokeStyle": color_map[l]})

/root/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:11: FutureWarning: by argument to sort_index is deprecated, pls use .sort_values(by=...)


In [107]:
f = file('bus/line.js', 'w')
f.write("var data=\n")
f.write(json.dumps(result))
f.close()

from IPython.display import FileLink
FileLink('bus/line.js')

/home/zhli7/bus/bus/line.js

## Sample

In [108]:
result = []
sample = clustered[clustered.cluster.isin(top_clusters[:top_count])]

for i in sample.index:
    label = sample.ix[i]['cluster']
    color = color_map[label]
    oc = sample.ix[i]['oc']
    dc = sample.ix[i]['dc']
    uv = sample.ix[i]['uv']
    result.append({"geometry":{"type":"LineString","coordinates":
                [[float(c) for c in oc.split(',')], [float(c) for c in dc.split(',')]]},
#                    "count":color,
                   "strokeStyle": color})

f = file('bus/line.js', 'w')
f.write("var data=\n")
f.write(json.dumps(result))
f.close()